# 🏆 FIFA World Cup 2026 - Brazil Win Simulation

## Scenario
This notebook simulates a specific trading scenario where:
- **Trader 1**: Buys $10 worth of Brazil YES at 51¢
- **Trader 2**: Buys $10 worth of Brazil YES at 52¢
- **Trader 3**: Buys $10 worth of Brazil YES at 53¢
- **Trader 4**: Buys $20 worth of France YES at 51¢

**Result**: Brazil wins the World Cup

We'll calculate:
- Shares each trader receives
- Final balances after resolution
- Profit/Loss for each trader
- ROI calculations

## Setup

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Market configuration
COUNTRIES = ['Argentina', 'Brazil', 'France', 'England', 'Spain']
INITIAL_BALANCE = 1000.00  # Each trader starts with $1000
WINNER_FEE_BPS = 400  # 4% fee (2% platform + 2% creator)
WINNER = 'Brazil'  # Resolution outcome

## Initial State

Each trader starts with $1,000 USDC

In [2]:
# Initialize trader balances
traders = {
    'Trader 1': {
        'cash': INITIAL_BALANCE,
        'shares': {'Brazil_YES': 0, 'France_YES': 0}
    },
    'Trader 2': {
        'cash': INITIAL_BALANCE,
        'shares': {'Brazil_YES': 0, 'France_YES': 0}
    },
    'Trader 3': {
        'cash': INITIAL_BALANCE,
        'shares': {'Brazil_YES': 0, 'France_YES': 0}
    },
    'Trader 4': {
        'cash': INITIAL_BALANCE,
        'shares': {'Brazil_YES': 0, 'France_YES': 0}
    }
}

print("📊 Initial Balances")
print("=" * 50)
for trader, data in traders.items():
    print(f"{trader}: ${data['cash']:.2f} USDC")

📊 Initial Balances
Trader 1: $1000.00 USDC
Trader 2: $1000.00 USDC
Trader 3: $1000.00 USDC
Trader 4: $1000.00 USDC


## Trading Activity

In [3]:
# Define trades
trades = [
    {'trader': 'Trader 1', 'market': 'Brazil', 'outcome': 'YES', 'price': 0.51, 'cost': 10.00},
    {'trader': 'Trader 2', 'market': 'Brazil', 'outcome': 'YES', 'price': 0.52, 'cost': 10.00},
    {'trader': 'Trader 3', 'market': 'Brazil', 'outcome': 'YES', 'price': 0.53, 'cost': 10.00},
    {'trader': 'Trader 4', 'market': 'France', 'outcome': 'YES', 'price': 0.51, 'cost': 20.00},
]

print("\n💰 Executing Trades")
print("=" * 80)
print(f"{'Trader':<12} {'Market':<10} {'Outcome':<8} {'Price':<8} {'Cost':<10} {'Shares'}")
print("-" * 80)

for trade in trades:
    trader = trade['trader']
    market = trade['market']
    outcome = trade['outcome']
    price = trade['price']
    cost = trade['cost']
    
    # Calculate shares received
    shares = cost / price
    
    # Update trader balance
    traders[trader]['cash'] -= cost
    share_key = f"{market}_{outcome}"
    traders[trader]['shares'][share_key] = traders[trader]['shares'].get(share_key, 0) + shares
    
    print(f"{trader:<12} {market:<10} {outcome:<8} ${price:<7.2f} ${cost:<9.2f} {shares:.2f}")

print("\n✅ All trades executed")


💰 Executing Trades
Trader       Market     Outcome  Price    Cost       Shares
--------------------------------------------------------------------------------
Trader 1     Brazil     YES      $0.51    $10.00     19.61
Trader 2     Brazil     YES      $0.52    $10.00     19.23
Trader 3     Brazil     YES      $0.53    $10.00     18.87
Trader 4     France     YES      $0.51    $20.00     39.22

✅ All trades executed


## Balances After Trading

In [4]:
print("\n📈 Balances After Trading")
print("=" * 80)
print(f"{'Trader':<12} {'Cash':<15} {'Brazil YES':<15} {'France YES':<15}")
print("-" * 80)

for trader, data in traders.items():
    brazil_yes = data['shares'].get('Brazil_YES', 0)
    france_yes = data['shares'].get('France_YES', 0)
    print(f"{trader:<12} ${data['cash']:<14.2f} {brazil_yes:<15.2f} {france_yes:<15.2f}")


📈 Balances After Trading
Trader       Cash            Brazil YES      France YES     
--------------------------------------------------------------------------------
Trader 1     $990.00         19.61           0.00           
Trader 2     $990.00         19.23           0.00           
Trader 3     $990.00         18.87           0.00           
Trader 4     $980.00         0.00            39.22          


## 🏆 Market Resolution: Brazil Wins!

When Brazil wins:
- **Brazil YES** tokens → $1.00 each (minus 4% fee)
- **Brazil NO** tokens → $0.00
- **All other countries (Argentina, France, England, Spain)**:
  - YES tokens → $0.00
  - NO tokens → $1.00 each (minus 4% fee)

### Fee Distribution (4% total)
- Platform (Nostra): 2%
- Market Creator (Jay): 2%

In [5]:
print(f"\n🏆 WORLD CUP 2026 FINAL RESULT: {WINNER} WINS!")
print("=" * 80)

# Track payouts
payouts = []
total_fees_platform = 0
total_fees_creator = 0

print(f"\n💰 Payout Distribution:")
print("=" * 80)
print(f"{'Trader':<12} {'Shares':<12} {'Gross Payout':<15} {'Fee (4%)':<12} {'Net Payout':<12}")
print("-" * 80)

# Pay Brazil YES holders
for trader, data in traders.items():
    brazil_yes_shares = data['shares'].get('Brazil_YES', 0)
    
    if brazil_yes_shares > 0:
        gross_payout = brazil_yes_shares * 1.0  # Each share worth $1
        total_fee = gross_payout * (WINNER_FEE_BPS / 10000)
        net_payout = gross_payout - total_fee
        
        # Split fee between platform and creator
        fee_platform = total_fee * 0.5
        fee_creator = total_fee * 0.5
        
        total_fees_platform += fee_platform
        total_fees_creator += fee_creator
        
        # Add payout to trader's cash
        traders[trader]['cash'] += net_payout
        
        payouts.append({
            'trader': trader,
            'market': 'Brazil YES',
            'shares': brazil_yes_shares,
            'gross': gross_payout,
            'fee': total_fee,
            'net': net_payout
        })
        
        print(f"{trader:<12} {brazil_yes_shares:<12.2f} ${gross_payout:<14.2f} ${total_fee:<11.2f} ${net_payout:<11.2f}")

# France YES holders get $0 (they lost)
print("\n❌ France YES holders: $0.00 (Brazil won, not France)")
for trader, data in traders.items():
    france_yes_shares = data['shares'].get('France_YES', 0)
    if france_yes_shares > 0:
        print(f"   {trader}: {france_yes_shares:.2f} France YES shares → $0.00")


🏆 WORLD CUP 2026 FINAL RESULT: Brazil WINS!

💰 Payout Distribution:
Trader       Shares       Gross Payout    Fee (4%)     Net Payout  
--------------------------------------------------------------------------------
Trader 1     19.61        $19.61          $0.78        $18.82      
Trader 2     19.23        $19.23          $0.77        $18.46      
Trader 3     18.87        $18.87          $0.75        $18.11      

❌ France YES holders: $0.00 (Brazil won, not France)
   Trader 4: 39.22 France YES shares → $0.00


## Final Balances & P&L

In [6]:
print("\n📊 Final Results")
print("=" * 80)
print(f"{'Trader':<12} {'Initial':<12} {'Final Cash':<15} {'Spent':<12} {'P&L':<12} {'ROI'}")
print("-" * 80)

results = []
for trader, data in traders.items():
    initial = INITIAL_BALANCE
    final = data['cash']
    spent = initial - (final - sum(
        payout['net'] for payout in payouts if payout['trader'] == trader
    ))
    pnl = final - initial
    roi = (pnl / initial) * 100
    
    results.append({
        'trader': trader,
        'initial': initial,
        'final': final,
        'spent': spent,
        'pnl': pnl,
        'roi': roi
    })
    
    pnl_symbol = '↑' if pnl >= 0 else '↓'
    pnl_color = '🟢' if pnl >= 0 else '🔴'
    
    print(f"{trader:<12} ${initial:<11.2f} ${final:<14.2f} ${spent:<11.2f} {pnl_color} ${pnl:<10.2f} {roi:>6.2f}%")

# Create DataFrame for better visualization
df_results = pd.DataFrame(results)
print("\n")
print(df_results.to_string(index=False))


📊 Final Results
Trader       Initial      Final Cash      Spent        P&L          ROI
--------------------------------------------------------------------------------
Trader 1     $1000.00     $1008.82        $10.00       🟢 $8.82         0.88%
Trader 2     $1000.00     $1008.46        $10.00       🟢 $8.46         0.85%
Trader 3     $1000.00     $1008.11        $10.00       🟢 $8.11         0.81%
Trader 4     $1000.00     $980.00         $20.00       🔴 $-20.00      -2.00%


  trader  initial       final  spent        pnl       roi
Trader 1   1000.0 1008.823529   10.0   8.823529  0.882353
Trader 2   1000.0 1008.461538   10.0   8.461538  0.846154
Trader 3   1000.0 1008.113208   10.0   8.113208  0.811321
Trader 4   1000.0  980.000000   20.0 -20.000000 -2.000000


## Fee Summary

In [7]:
print("\n💸 Fee Distribution")
print("=" * 80)
print(f"Platform (Nostra):  ${total_fees_platform:.2f} (2%)")
print(f"Creator (Jay):      ${total_fees_creator:.2f} (2%)")
print(f"Total Fees:         ${total_fees_platform + total_fees_creator:.2f} (4%)")


💸 Fee Distribution
Platform (Nostra):  $1.15 (2%)
Creator (Jay):      $1.15 (2%)
Total Fees:         $2.31 (4%)


## Summary Statistics

In [8]:
winners = [r for r in results if r['pnl'] > 0]
losers = [r for r in results if r['pnl'] < 0]
break_even = [r for r in results if r['pnl'] == 0]

print("\n📈 Market Statistics")
print("=" * 80)
print(f"Total Participants:  {len(results)}")
print(f"🟢 Winners:          {len(winners)}")
print(f"🔴 Losers:           {len(losers)}")
print(f"➖ Break Even:       {len(break_even)}")
print(f"")
print(f"Total Volume:        ${sum(trade['cost'] for trade in trades):.2f}")
print(f"Avg ROI:             {np.mean([r['roi'] for r in results]):.2f}%")
print(f"Best ROI:            {max([r['roi'] for r in results]):.2f}% ({max(results, key=lambda x: x['roi'])['trader']})")
print(f"Worst ROI:           {min([r['roi'] for r in results]):.2f}% ({min(results, key=lambda x: x['roi'])['trader']})")


📈 Market Statistics
Total Participants:  4
🟢 Winners:          3
🔴 Losers:           1
➖ Break Even:       0

Total Volume:        $50.00
Avg ROI:             0.13%
Best ROI:            0.88% (Trader 1)
Worst ROI:           -2.00% (Trader 4)


## Key Insights

### Winners
- **Traders 1, 2, 3** bet on Brazil and won
- They receive their shares back as $1.00 each, minus 4% fee
- Net return = 96% of shares value

### Losers
- **Trader 4** bet on France, which lost
- Their France YES tokens are worth $0.00
- Total loss = $20.00 (their entire investment)

### Price Impact on Profit
- **Trader 1** (bought at 51¢): Best profit margin
- **Trader 2** (bought at 52¢): Slightly less profit
- **Trader 3** (bought at 53¢): Lowest profit among winners
- Lower entry price = Higher profit (for correct prediction)